In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', save_generated_data=False)
data.generate_data('letters_word_accentuation_train',
                   'letters_word_accentuation_test',
                   'letters_word_accentuation_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [6]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')

print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_train[pos])
print(data.decode_y(data.y_train[pos]))
print(dictionary)

(482209, 23, 36)
(52058, 23, 36)
(54222, 23, 36)


NameError: name 'dictionary' is not defined

In [4]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [5]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20




conv_input_shape=(23, 36)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [6]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              verbose=2)

Epoch 1/400
15s - loss: 0.2139 - actual_accuracy: 0.3863
Epoch 2/400
17s - loss: 0.1541 - actual_accuracy: 0.5609
Epoch 3/400
17s - loss: 0.1379 - actual_accuracy: 0.6217
Epoch 4/400
17s - loss: 0.1277 - actual_accuracy: 0.6658
Epoch 5/400
20s - loss: 0.1199 - actual_accuracy: 0.6924
Epoch 6/400
24s - loss: 0.1136 - actual_accuracy: 0.7060
Epoch 7/400
22s - loss: 0.1074 - actual_accuracy: 0.7287
Epoch 8/400
22s - loss: 0.1018 - actual_accuracy: 0.7439
Epoch 9/400
22s - loss: 0.1001 - actual_accuracy: 0.7490
Epoch 10/400
22s - loss: 0.0946 - actual_accuracy: 0.7652
Epoch 11/400
23s - loss: 0.0927 - actual_accuracy: 0.7710
Epoch 12/400
18s - loss: 0.0895 - actual_accuracy: 0.7818
Epoch 13/400
18s - loss: 0.0857 - actual_accuracy: 0.7888
Epoch 14/400
18s - loss: 0.0812 - actual_accuracy: 0.8044
Epoch 15/400
17s - loss: 0.0795 - actual_accuracy: 0.8059
Epoch 16/400
17s - loss: 0.0760 - actual_accuracy: 0.8134
Epoch 17/400
18s - loss: 0.0739 - actual_accuracy: 0.8186
Epoch 18/400
18s - loss

22s - loss: 0.0148 - actual_accuracy: 0.9672
Epoch 143/400
21s - loss: 0.0145 - actual_accuracy: 0.9672
Epoch 144/400
21s - loss: 0.0145 - actual_accuracy: 0.9672
Epoch 145/400
21s - loss: 0.0129 - actual_accuracy: 0.9708
Epoch 146/400
20s - loss: 0.0134 - actual_accuracy: 0.9702
Epoch 147/400
21s - loss: 0.0131 - actual_accuracy: 0.9718
Epoch 148/400
20s - loss: 0.0127 - actual_accuracy: 0.9735
Epoch 149/400
20s - loss: 0.0138 - actual_accuracy: 0.9694
Epoch 150/400
21s - loss: 0.0129 - actual_accuracy: 0.9715
Epoch 151/400
21s - loss: 0.0138 - actual_accuracy: 0.9703
Epoch 152/400
21s - loss: 0.0130 - actual_accuracy: 0.9718
Epoch 153/400
21s - loss: 0.0129 - actual_accuracy: 0.9713
Epoch 154/400
21s - loss: 0.0135 - actual_accuracy: 0.9702
Epoch 155/400
21s - loss: 0.0124 - actual_accuracy: 0.9732
Epoch 156/400
21s - loss: 0.0119 - actual_accuracy: 0.9745
Epoch 157/400
21s - loss: 0.0124 - actual_accuracy: 0.9728
Epoch 158/400
21s - loss: 0.0145 - actual_accuracy: 0.9682
Epoch 159/4

23s - loss: 0.0102 - actual_accuracy: 0.9788
Epoch 282/400
21s - loss: 0.0079 - actual_accuracy: 0.9836
Epoch 283/400
21s - loss: 0.0085 - actual_accuracy: 0.9816
Epoch 284/400
21s - loss: 0.0074 - actual_accuracy: 0.9835
Epoch 285/400
21s - loss: 0.0076 - actual_accuracy: 0.9833
Epoch 286/400
22s - loss: 0.0080 - actual_accuracy: 0.9832
Epoch 287/400
21s - loss: 0.0070 - actual_accuracy: 0.9845
Epoch 288/400
21s - loss: 0.0074 - actual_accuracy: 0.9844
Epoch 289/400
21s - loss: 0.0078 - actual_accuracy: 0.9834
Epoch 290/400
21s - loss: 0.0073 - actual_accuracy: 0.9837
Epoch 291/400
21s - loss: 0.0078 - actual_accuracy: 0.9843
Epoch 292/400
22s - loss: 0.0074 - actual_accuracy: 0.9830
Epoch 293/400
21s - loss: 0.0075 - actual_accuracy: 0.9828
Epoch 294/400
22s - loss: 0.0080 - actual_accuracy: 0.9833
Epoch 295/400
21s - loss: 0.0069 - actual_accuracy: 0.9844
Epoch 296/400
21s - loss: 0.0069 - actual_accuracy: 0.9859
Epoch 297/400
21s - loss: 0.0077 - actual_accuracy: 0.9831
Epoch 298/4

In [7]:
name = '20_test_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [4]:
#data = Data('l')
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()

In [6]:
accented_vowels

['à', 'á', 'ä', 'é', 'ë', 'ì', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']